## Hindsight Bias Calculation

This code calculates the ex-post hindsight bias for the online bin packing instance, by computing $\max_s \Delta_t^\dagger(s)$.  To adjust the parameters and recover the experiments in the paper, modify the STEP_LIMIT parameter in the binpacking environment below.

In [ ]:
import numpy as np
import guidedrl
from guidedrl.bin_packing.binpacking import BinPackingLW1, BinPackingEnv, BinPackingToy, BIG_NEG_REWARD
from guidedrl.bin_packing.optimal_bin_packing import BinPackingILP
from guidedrl.bin_packing.common import get_exogenous_dataset, get_optimal_policy, get_ip_values
import pulp
import itertools

DEBUG = False

In [ ]:
BIN_CAPACITY = 5
ITEM_SIZES = [2,3]
ITEM_PROBS = [0.8, 0.2]
STEP_LIMIT = 5

The goal of this notebook is to understand, in a toy bin packing problem, how the 'optimal' actions are preserved via the Q^IP solution versus the Q^star solution.  The main aim is to calculate differences of the following:

$\hat{Q}(s,a') - Q^\star(s,a') + Q^\star(s, a^\star) - \hat{Q}(s,a^\star)$

where $a' = \arg \max_a \hat{Q}(s,a)$ and $a^\star = \arg \max_a Q^\star(s,a)$, and $\hat{Q}$ is the expectation of $Q^{IP}(s,a)$ over future exogenous sequences sampled independently from their respective distribution.

Note that this arises from noticing that for any policy $\mu$ we have that:
$V^\mu - V^\pi = E_{d_\pi}[Q^\mu(s, a^\star) - Q^\mu(s,a')]$ and adding and subtracting terms.

We start by setting up the simple toy bin packing environment via the OpenAI Gym interface.

In [ ]:
env = BinPackingEnv()
env.bin_capacity = BIN_CAPACITY
env.item_sizes = ITEM_SIZES
env.item_probs = ITEM_PROBS
env.step_limit = STEP_LIMIT
env._build_obs_space()
env.reset()


The following code solves for the optimal $Q^*$ and $Q^\dagger$ values in the bin packing environment.

In [ ]:
vStar, qStar = get_optimal_policy(env) # gets the optimal policy and IP values
vIP, qIP = get_ip_values(env, num_iters = 100)

Next we compute the number of times the resulting policies are different across the state space, alongwith the maximum difference in the hindsight bias terms.

In [ ]:
state_limits = np.copy(env.observation_space.high+1)
same_count = 0
diff_count = 0
differences =  np.zeros(np.append(state_limits, env.step_limit))
for h in np.arange(env.step_limit - 1, -1, -1):
    print(f'Step: {h}')
    for s in itertools.product(*[np.arange(x) for x in state_limits]):
        if np.sum(list(s)[:-1]) <= h and list(s)[-1] in env.item_sizes: # in a valid state
            ahat = np.argmax(qIP[s+(h,)])
            astar = np.argmax(qStar[s+(h,)])
            if ahat == astar:
                same_count += 1
            else:
                diff_count += 1
            s_hat = s+(h,)+(ahat,)
            s_star = s+(h,)+(astar,)
            differences[s+(h,)] = qIP[s_hat] - qStar[s_hat] + qStar[s_star] - qIP[s_star]
print(f'Same count: {same_count} and different count: {diff_count}')
print(f'Maximum difference: {np.max(differences)}')